In [1]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
from tqdm import tqdm
import pandas as pd
from rank_bm25 import BM25Okapi 
import ir_datasets
import random

## Collecting Data

In [2]:
dataset = ir_datasets.load("neuclir/1/multi/trec-2023")

In [3]:
documents_df = pd.read_csv('sbert_documents.csv')
qrels_df = pd.DataFrame(dataset.qrels_iter())
queries_df = pd.DataFrame(dataset.queries_iter())
qrels_doc_ids = set(qrels_df["doc_id"])

In [4]:
documents_df.head()

,id,title,content,title_embedding,content_embedding
0,7707eaec-e3a6-422a-9e49-68ef7d2baaec,健康：身體和精神健康相互作用和影響背後的原理,健康：身體和精神健康相互作用和影響背後的原理\n\n10 分钟前\n\n圖像來源，Getty...,"tensor([ 1.7091e-01, 1.1061e-01, 7.6357e-03,...","tensor([ 0.2471, -0.0563, 0.1989, 0.2372, 0..."
1,27a8ca18-15fa-4f80-af10-d0fa53968995,尼羅河水爭奪戰越演越烈 聯合國安理會將召開會議,埃塞俄比亞復興大壩第二期蓄水工程遭到藍尼羅河下游國家蘇丹與埃及的強烈反對，聯合國安理會在突尼...,"tensor([ 1.7681e-01, 1.0508e-01, -6.0566e-02,...","tensor([-0.1660, 0.1614, -0.0299, -0.0677, 0..."
2,2e7d5130-d497-4f8c-9132-a4dc22ca6ce8,Visa：旗下虛擬貨幣卡上半年使用量逾10億美元,Visa Inc.財務總監Vasant Prabhu周三(7日)表示，今年上半年，旗下虛擬貨...,"tensor([ 1.7375e-01, 2.0704e-02, -2.7007e-01,...","tensor([ 1.2213e-01, -5.1399e-02, -1.6382e-01,..."
3,fdc63781-a14a-47c5-8518-8774df8d0e51,大蛇潛馬桶 偷襲如廁男,奧地利一名六十五歲男子在家中上廁所時，突遭到一條一點六米長的蟒蛇咬傷下體，所幸只受到輕傷，沒...,"tensor([-0.1478, 0.1569, 0.1280, -0.1398, -0...","tensor([-1.6577e-01, 1.3552e-01, 1.7160e-01,..."
4,f55f74b2-433b-478f-9b1b-158eb7cfa783,野火烟雾缭绕，BC 省专家支招如何在糟糕空气中保护自己,BC 省南部内陆大部分地区已经发出空气质量警告，而且 BC 省的火灾风险一直处于高位。\n\...,"tensor([ 0.0198, 0.4940, -0.1451, 0.1459, 0...","tensor([ 0.0918, 0.2722, 0.0957, 0.1512, 0..."


In [5]:
queries_df.head()

,query_id,title,description,narrative,fa_mt_title,fa_mt_description,fa_mt_narrative,ru_mt_title,ru_mt_description,ru_mt_narrative,zh_mt_title,zh_mt_description,zh_mt_narrative
0,200,Corruption Bribery Sports Federation Olympics,Are there cases of Institutional Corruption an...,We are Looking for articles that contain a cas...,المپیک فدراسیون ورزشی رشوه خواری فساد,آیا مواردی از فساد نهادی و رشوه خواری در ورزش ...,ما به دنبال مقالاتی هستیم که حاوی یک مورد فساد...,Олимпийские игры Федерации спортивных игр корр...,Существуют ли случаи институциональной коррупц...,"Мы ищем статьи, которые содержат случай финанс...",腐败贿赂体育联合会奥运会,体育中是否存在机构腐败和贿赂的案例？,我们正在寻找在机构层面上包含金融腐败或贿赂案件的文章。奥运会和体育联合会中的贿赂案件是相关的...
1,201,China investment in Iran,In what fields or industries China’s investmen...,We need articles that specify at least one fie...,سرمایه گذاری چین در ایران,در کدام زمینه ها یا صنایع سرمایه گذاری چین در ...,ما به مقالاتی نیاز داریم که حداقل یک زمینه سرم...,Китайские инвестиции в Иран,В каких областях или промышленности инвестиции...,"Нам нужны статьи, в которых указано хотя бы од...",中国对伊朗的投资,在哪些领域或行业中，中国对伊朗的投资是？,我们需要至少指定一个投资领域的文章，无论是皮带和道路倡议，石油，水坝等。货币的投资金额不提及...
2,202,"Emerging technologies, precision farming, smar...",What are some of the latest emerging technolog...,Find articles on emerging technologies in prec...,فن آوری های نوظهور ، کشاورزی دقیق ، کشاورزی هو...,برخی از جدیدترین فن آوری های نوظهور که صنعت کش...,مقالاتی در مورد فن آوری های نوظهور در کشاورزی ...,"Новые технологии, точное сельское хозяйство, у...",Каковы некоторые из последних новых технологий...,Найдите статьи о новых технологиях в области т...,新兴技术，精确农业，智能农业，农业,哪些最新的新兴技术正在改变农业行业？,查找有关精确农业和农业中新兴技术的文章，这些文章提供了这些技术如何用于提高农作物产量并提高农...
3,203,Ever Given's Stuck,Find information about the ship Ever Given bei...,Find articles on the event when the container ...,تا به حال گیر کرده است,اطلاعات مربوط به کشتی را که در کانال سوئز گیر ...,مقالاتی را در مورد این رویداد پیدا کنید که کشت...,Когда -либо дал застрял,"Найдите информацию о корабле, когда -либо заст...","Найдите статьи на мероприятии, когда контейнер...",曾经被卡住了,查找有关被困在苏伊士运河的船的信息,在2021年春季遇到的集装箱船被困在苏伊士运河中时，请查找有关事件的事件的文章。包括有关事件...
4,204,Penalty doping sports stop,What strategies are used to reduce/stop doping...,Any mention of the monetary penalties or exclu...,مجازات دوپینگ ورزش توقف,از چه استراتژی هایی برای کاهش/متوقف کردن دوپین...,هرگونه ذکر مجازات های پولی یا محرومیت تیم ها ی...,Спортивная остановка пенальти,Какие стратегии используются для уменьшения/ос...,Любое упоминание о денежных наказаниях или иск...,点球兴奋剂运动停止,哪些策略用于减少运动中的掺杂？,包括任何提及货币罚款或排除在团队中或夺回奖牌的罚款。接受掺杂的测试将不被视为预防措施。


## Preprocessing

In [6]:
documents_df['content'] = documents_df['content'].fillna('') 
documents_df = documents_df[documents_df['content'].str.strip() != '']

def detect_script(text):
    # Check if text contains mostly Cyrillic characters (for Russian)
    if any('\u0400' <= char <= '\u04FF' for char in text):
        return "rus"  # ISO 639-2 code for Russian
    # Check if text contains Chinese characters
    elif any('\u4E00' <= char <= '\u9FFF' for char in text):
        return "zho"  # ISO 639-2 code for Chinese
    # Check if text contains Arabic script (used in Persian)
    elif any('\u0600' <= char <= '\u06FF' for char in text):
        return "fas"  # ISO 639-2 code for Persian
    else:
        return "unknown"

# Apply the detection function
documents_df['language'] = documents_df['content'].apply(detect_script)
documents_df = documents_df[documents_df['language'] != 'unknown']

print(documents_df['language'].value_counts())

# Group documents by detected language
documents_by_language = {
    lang: docs.reset_index(drop=True)
    for lang, docs in documents_df.groupby('language')
}

language
zho    26572
fas    25315
rus    24951
Name: count, dtype: int64


In [7]:
documents_df[documents_df['language'] == 'fas']

,id,title,content,title_embedding,content_embedding,language
26595,45ed98fb-30ba-4e28-9d10-2dcb5efa705e,"تلاش برای توقفِ ""سنت وحشیانه قتل‌عام نهنگ‌ها"" ...","در عرض ۲۴ ساعت در جزایر ""فارو"" ۱۳۱ قطعه نهنگ خ...","tensor([ 3.7155e-02, 2.1960e-01, 3.7627e-02,...","tensor([ 6.5190e-02, -9.5028e-02, 2.2075e-01,...",fas
26596,01bbfede-5dcb-499d-9c92-34a8f6ae57cc,با امضای قرارداد جبران خسارت، اور‌گیون سرانجام...,با امضای قرارداد جبران خسارت، اور‌گیون سرانجام...,"tensor([ 7.9379e-02, 2.3922e-02, 9.2551e-02,...","tensor([ 1.7791e-01, 3.3269e-02, 1.0767e-01,...",fas
26597,a62c5bf8-ff3e-4ef8-90e3-77a748cb7e67,افقری: از قبل به ما اعتماد نشد!/ هدفم مدال باز...,شناگر ایرانی در پاسخ به اینکه، چرا با وجودی که...,"tensor([-1.3324e-02, -2.4414e-02, -1.3641e-01,...","tensor([ 4.0034e-02, 1.8567e-01, -2.4527e-02,...",fas
26598,9b0a2f83-be93-43c5-9891-aaf148c229b3,مراسم معارفه سرپرست فدراسیون اسکیت برگزار شد,به گزارش ایرنا، مراسم تودیع و معارفه رییس پیشی...,"tensor([-9.1050e-02, 5.8356e-01, -1.3420e-01,...","tensor([ 0.0534, 0.3432, -0.0467, -0.1052, -0...",fas
26599,53b05cc3-a95a-4739-b764-41fcfaedd734,قیمت میلگرد چگونه محاسبه می شود؟,امروزه از میلگرد ساده و آجدار در صنایع مختلف ا...,"tensor([ 9.7606e-02, 2.6175e-01, 7.7978e-02,...","tensor([-1.5112e-01, 2.6945e-01, 1.2211e-01,...",fas
...,...,...,...,...,...,...
52037,99a7b445-7766-4af1-973e-e29662e88be0,فرهادی زاد: بانوان ورزشکار کشورمان با کسب ۱۱ س...,به گزارش به گزارش خبرنگار گروه ورزشی باشگاه خب...,"tensor([-3.0691e-02, 1.8205e-01, -4.9377e-02,...","tensor([ 0.0012, 0.2914, -0.0494, 0.1155, -0...",fas
52038,c3a8bba5-ca34-47d6-a165-97e4b7b0005a,توییت‌های جنجالی و ادامه دار ایلان ماسک؛ اجرای...,به گزارش خبرنگار حوزه دریچه فناوری گروه فضای م...,"tensor([-1.6474e-01, 1.0562e-01, 8.3292e-02,...","tensor([-1.4413e-01, 2.1819e-01, -7.1567e-03,...",fas
52039,3b378632-f441-435b-ac0b-ad447f0034d3,بازار، بدون درک «انقلاب صنعتی چهارم» آینده‌ای ...,به گزارش خبرنگار مهر، کتاب«اینترنت اشیا» به قل...,"tensor([-0.0081, 0.1316, -0.0555, -0.3350, 0...","tensor([-0.0911, -0.0161, -0.0424, -0.1205, 0...",fas
52040,a6b314e3-92f6-45b4-bbe4-f4e359231390,تشکیک در صلحبانی سازمان ملل در آفریقا,آنها در اعتراض به ناکارآمدی این نیروهای به اصط...,"tensor([-1.6679e-01, 3.5203e-01, -2.8482e-01,...","tensor([-2.5236e-01, 2.2948e-01, -1.4293e-01,...",fas


In [8]:
# getting title query 
query_languages = ['fa', 'ru', 'zh']
queries_multilingual = {lang: queries_df[['query_id', f'{lang}_mt_title']].rename(columns={f'{lang}_mt_title': 'query_text'}) for lang in query_languages}
key_mapping = {'fa': 'fas', 'ru': 'rus', 'zh': 'zho'}
queries_multilingual = {key_mapping.get(k, k): v for k, v in queries_multilingual.items()}

In [9]:
queries_multilingual

{'fas':    query_id                                         query_text
 0       200              المپیک فدراسیون ورزشی رشوه خواری فساد
 1       201                          سرمایه گذاری چین در ایران
 2       202  فن آوری های نوظهور ، کشاورزی دقیق ، کشاورزی هو...
 3       203                             تا به حال گیر کرده است
 4       204                            مجازات دوپینگ ورزش توقف
 ..      ...                                                ...
 71      271                                     بازآرایی نماتد
 72      272                 Hanyu Yuzuru ، اسکیت شکل ، دستاورد
 73      273                               حمله سفارت ایران وین
 74      274              فاجعه هسته ای فوکوشیما ده سال بازسازی
 75      275                                انتخاب شناگر المپیک
 
 [76 rows x 2 columns],
 'rus':    query_id                                         query_text
 0       200  Олимпийские игры Федерации спортивных игр корр...
 1       201                        Китайские инвестиции в Иран


## Ranking Documents using BM25

In [10]:
bm25_results = {}

for query_lang, query_df in queries_multilingual.items():
    print(f"Processing BM25 for queries in {query_lang}...")

    # Select the corresponding document set
    if query_lang in documents_by_language:
        docs = documents_by_language[query_lang]
        doc_texts = docs['content'].tolist()
        doc_ids = docs['id'].tolist()

        bm25 = BM25Okapi([doc.split() for doc in doc_texts])  # Tokenize documents

        # Process each query
        for _, row in tqdm(query_df.iterrows(), total=len(query_df), desc=f"Retrieving for {query_lang}"):
            query_id = row['query_id']
            query_text = row['query_text'] # title
            
            # Tokenize query
            query_tokens = query_text.split()
            
            scores = bm25.get_scores(query_tokens)
            
            top_n = 10
            top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n]
            top_docs = [(doc_ids[i], scores[i]) for i in top_indices]

            # Store results
            bm25_results[(query_id, query_lang)] = top_docs

Processing BM25 for queries in fas...


Retrieving for fas: 100%|██████████| 76/76 [00:03<00:00, 23.31it/s]


Processing BM25 for queries in rus...


Retrieving for rus: 100%|██████████| 76/76 [00:02<00:00, 26.55it/s]


Processing BM25 for queries in zho...


Retrieving for zho: 100%|██████████| 76/76 [00:00<00:00, 146.92it/s]


In [11]:
bm25_results_df = pd.DataFrame([
    {"query_id": query_id, "query_lang": query_lang, "doc_id": doc_id, "score": score}
    for (query_id, query_lang), docs in bm25_results.items()
    for doc_id, score in docs
])

In [12]:
print(bm25_results_df.groupby('query_lang')['query_id'].nunique())
bm25_results_df.head()

query_lang
fas    76
rus    76
zho    76
Name: query_id, dtype: int64


,query_id,query_lang,doc_id,score
0,200,fas,61dbcac1-39da-48a1-ba00-ef73d74e5365,31.712739
1,200,fas,6beb3075-70e0-40a4-8968-f88165874340,29.415557
2,200,fas,874c32a9-70b5-46ae-bbe0-35470131003e,28.469044
3,200,fas,0994d767-c47e-45f9-9150-f013f1b05cc1,28.303887
4,200,fas,de8bb4aa-a22f-4c47-9470-ca75a674b489,28.115222


## Evaluation

In [13]:
# filtered qrels dataset for evaluation 
valid_doc_ids = set(documents_df['id'])
qrels_df = pd.DataFrame(dataset.qrels_iter())
print(qrels_df.shape[0])

79934


In [14]:
qrels_df

,query_id,doc_id,relevance,iteration
0,200,00258365-6d48-49cc-901f-578b883c8226,1,fas
1,200,010fd82c-4423-41a6-ac56-4d036ccf0524,2,fas
2,200,013c3243-d1b5-47e6-bcf5-26c092ac9ff5,0,fas
3,200,01aed8a4-f1b9-4729-97f0-525338029268,0,fas
4,200,01e59322-b610-4d44-9ecd-81bfad2f2b5e,0,fas
...,...,...,...,...
79929,275,fdb86fa5-9ec9-4452-84d9-da8df589ddc8,0,zho
79930,275,ff0ad949-3ca1-4c0e-ab90-e8a507862c37,0,zho
79931,275,ff1f235f-fe32-4455-9206-251ee03bd1e1,0,zho
79932,275,ff473b84-85b3-4314-8308-eea5590eca19,0,zho


In [15]:
import ir_measures
from ir_measures import ScoredDoc, Qrel

qrels = [
    Qrel(query_id=f"{row['query_id']}",
         doc_id=row['doc_id'],
         relevance=row['relevance'])
    for _, row in qrels_df.iterrows()
]

results = [
    ScoredDoc(query_id=f"{row['query_id']}",
              doc_id=row['doc_id'],
              score=row['score'])
    for _, row in bm25_results_df.iterrows()
]


metrics = [
    ir_measures.nDCG @ 20,   # nDCG@20
    ir_measures.AP,          # Average Precision
    ir_measures.RBP(rel=1),
    ir_measures.R @ 100,     # Recall@100
    ir_measures.R @ 1000     # Recall@1000
]

evaluation = ir_measures.calc_aggregate(metrics, qrels, results)

print("Evaluation Metrics:")
for metric, value in evaluation.items():
    print(f"{metric}: {value:.4f}")


Evaluation Metrics:
AP: 0.0293
RBP(rel=1): 0.1700
R@1000: 0.0519
R@100: 0.0519
nDCG@20: 0.2240
